In [6]:
import pandas as pd
import glob
from funcLibrary import *
import numpy as np


claim_path = ".\After_Process\monthly_process\*"
company_path = ".\After_Process\Company\*"


list_claim = glob.glob(claim_path)
list_company = glob.glob(company_path)


first_list = ['2009', '2010', '2011']
last_list = ['2013', '2014', '2015']


first_claim_list = sorted([i for i in list_claim for j in first_list if j in i])
last_claim_list = sorted([i for i in list_claim for j in last_list if j in i])


first_company_list = sorted([i for i in list_company for j in first_list if j in i])
last_company_list = sorted([i for i in list_company for j in last_list if j in i])


first_df = get3Years(first_company_list, first_claim_list)
last_df = get3Years(last_company_list, last_claim_list)

first_group = first_df[['投保證號', '行業別細類(業別)']].rename({'行業別細類(業別)':'group'}, axis=1)
last_group = last_df[['投保證號', '行業別細類(業別)']].rename({'行業別細類(業別)':'group'}, axis=1)

# read the latest industry type
df_latest = pd.read_csv('.\After_Process\Company\Company-2015.txt', sep='\t')
df_latest = df_latest.drop_duplicates('投保證號').reset_index(drop=True)

first_group = replaceIndustryType(first_group, df_latest)
last_group = replaceIndustryType(last_group, df_latest)

# 6 years
first, last = getIntersection(first_df.copy(), last_df.copy())

# first = replaceOriginalIndustryType(first.copy(), './行業標準分類第九次修訂(edited).xlsx')

first['group'] = first['行業別細類(業別)']
first['行業別細類(業別)'] = first['group']
# first = first.drop('行業別細類(業別)', axis=1)

table = createDistanceTable(first)

first_=first.copy()
#def getLossOriginal(df_first, group, distanceTable):
"""
Lost Function
:param df_first: first 3 year claim , last 3 year claim
:param group: classifier group set
:return: Loss DataFrame
"""
groupSet = sorted(set(first_['group']))
lossList = []
for i in groupSet:
    index = int(i)
    cur = first_[first_['group'] == index]
    df_first_cur = first_[first_['投保證號'].isin(cur['投保證號'])].reset_index(drop=True)
    # print(df_first_cur)
    loss = []

    # Fill Zero
    new_df = fillZero(df_first_cur['行業別細類(業別)'].astype(int).astype(str), 4)

    #combi = combinations(new_df.unique(), 2)

    # Calculate Loss
    for j in range(0, len(df_first_cur)):
      loss.append(df_first_cur.loc[j, 'first_給付金額'] - df_first_cur.loc[j, 'last_給付金額'])
    #lossList.append(math.pow(sum(loss),2) )#測試拔掉distancetable

    lossList.append(math.pow(sum(loss),2) * calcDefault(new_df, table))



fitness_loss = math.fabs(sum(lossList))
  #return fitness_loss

#loss = getLossOriginal(first.copy(), first.copy(),table)



input total Company df Length = 457146
['.\\After_Process\\monthly_process\\2009.txt', '.\\After_Process\\monthly_process\\2010.txt', '.\\After_Process\\monthly_process\\2011.txt']
input total Claim df Length = 73379
output dataframe length = 457146
input total Company df Length = 495658
['.\\After_Process\\monthly_process\\2013.txt', '.\\After_Process\\monthly_process\\2014.txt', '.\\After_Process\\monthly_process\\2015.txt']
input total Claim df Length = 84334
output dataframe length = 495658
first確認交集後投保證號= 375259
last確認交集後投保證號= 375259
table created


In [56]:
#1.8131152683553178e+20

print('Original Loss        = {0:-<10}'.format(1.8131152683553178e+20))
print('Group:19 , epoch:260 = {0:-<10}'.format(2639912529253858304.000000))
print('Group:495, epoch:259 = {0:-<10}'.format(53934975224845017088.000000))


print('學弟原本計算Original  = {0:-<10}'.format(1.790964245832112e+20))
print('學弟原本計算Original  = {0:-<10}'.format(160863728693428125696.000000))

Original Loss        = 1.8131152683553178e+20
Group:19 , epoch:260 = 2.6399125292538583e+18
Group:495, epoch:259 = 5.393497522484502e+19
學弟原本計算Original  = 1.790964245832112e+20
學弟原本計算Original  = 1.6086372869342813e+20


In [33]:
print(len(groupSet))
first_.head()

594


,投保證號,實績費率,職災費率,行業別細類(業別),first_給付金額,last_給付金額,group
0,1000011,0.53,0.53,2420.0,5.503030e+04,18333.333333,2420.0
1,1000014,0.25,0.25,1610.0,3.316810e+06,161235.294118,1610.0
2,1000026,0.28,0.28,1013.0,0.000000e+00,81674.418605,1013.0
3,1000027,0.48,0.48,1540.0,1.281667e+05,0.000000,1540.0
4,1000029,0.22,0.22,3111.0,0.000000e+00,0.000000,3111.0


In [31]:
table.head()

,0111,0112,0113,0114,0115,0116,0117,0118,0119,0121,...,9691,9692,9693,9699,9811,9812,9820,9901,9902,9909
0111,1,2,2,2,2,2,2,2,2,4,...,8,8,8,8,8,8,8,8,8,8
0112,2,1,2,2,2,2,2,2,2,4,...,8,8,8,8,8,8,8,8,8,8
0113,2,2,1,2,2,2,2,2,2,4,...,8,8,8,8,8,8,8,8,8,8
0114,2,2,2,1,2,2,2,2,2,4,...,8,8,8,8,8,8,8,8,8,8
0115,2,2,2,2,1,2,2,2,2,4,...,8,8,8,8,8,8,8,8,8,8


In [34]:
fitness_loss

1.8131152683553178e+20

In [41]:
gg=calcDefault(new_df, table)
print(gg)
combi = combinations(new_df, 2)
total_distance = sumDistance(combi, table)
combination_count = combination(len(new_df), 2)
print(total_distance,combination_count)


1.0
1485 1485.0
